## Encrypt user private key with Fernet with encryption key stored in GCP KMS
https://cryptography.io/en/latest/fernet/

https://www.geeksforgeeks.org/fernet-symmetric-encryption-using-cryptography-module-in-python/ 


#### Step 1. Activate Cloud Key Management Service (KMS) API
https://console.cloud.google.com/marketplace/product/google/cloudkms.googleapis.com

#### Step 2. Create a Google service account that has the role of Cloud KMS Admin
https://console.cloud.google.com/iam-admin/serviceaccounts

Generate and download a private key for the service account in JSON format

#### Step 3. Install KMS client lib

pip install google-cloud-kms firebase-admin

#### Step 4. Connect to KMS

In [2]:
from google.cloud import kms
from google.oauth2 import service_account

In [5]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
KMS_SERVICE_ACCOUNT_FILE = '../../kms-admin.json'

project_id = 'wallet-login-45c1c'

In [8]:

# Create the credentials object from the service account file.
credentials = service_account.Credentials.from_service_account_file(KMS_SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
client = kms.KeyManagementServiceClient(credentials=credentials)


create a key ring, a key, and a key version. You can specify the name of the key when you create it. 
For example, you can use the following code to create a symmetric encryption key named gladius-key in a key ring named gladius-key-ring in the global location

In [ ]:


# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

# Create the key ring.
key_ring = {}
client.create_key_ring(request={"parent": parent, "key_ring_id": "gladius-key-ring", "key_ring": key_ring})

# Create the key.
purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
crypto_key = {"purpose": purpose}
client.create_crypto_key(request={"parent": key_ring_name, "crypto_key_id": "gladius-key", "crypto_key": crypto_key})


### Test KMS

You can use the client library to encrypt and decrypt data with the key you created. For example, you can use the following code to encrypt and decrypt a message using gladius-key

In [9]:

# Build the key name.
key_name = f"projects/{project_id}/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key"

# The message to encrypt.
message = b"Hello Gladius 2!"

# Encrypt the message.
response = client.encrypt(request={"name": key_name, "plaintext": message})
ciphertext = response.ciphertext

print(ciphertext)

b'\n$\x00\x04\xc9Y\x9b\rt\x94\xc1\xee]\x95;@Ek\xc0\xbcO\x14PC\xf4*!3\xb7\xb1\xc4D\xc8b`\xf5\x1aG\x129\x00\x06\x16!\xa1\xeeZg\xf2H\xc8O\t\x00Wv\xab\x9d\xd6\xf5 \xefR\xbd\xdbr\x17w\xd8^\xac \xb0\xfe\x82\x0f\xf6\xfc\xe2}\xee\xab\xb4Q\x16`\xca!\xb4h\xc9\xd0\xf3\x9a\xda\xedh'


In [10]:
# Decrypt the message.
response = client.decrypt(request={"name": key_name, "ciphertext": ciphertext})
plaintext = response.plaintext

print(plaintext)


b'Hello Gladius 2!'


### Step 5. Encrypt private_key with Fernet and encryption key stored in Cloud KMS
https://cryptography.io/en/latest/fernet/

In [11]:
# pip install cryptography

from cryptography.fernet import Fernet

Use the generate_key() method of the Fernet class to generate a new fernet key2. The key is a random value, and will be completely different each time you call the method2. The key is a URL-safe base64-encoded 32-byte key2. You need to keep this key safe and secret, as it is the only way to encrypt and decrypt your data with Fernet2. For example:

In [12]:
# this re-gerate the encryption key

key = Fernet.generate_key()

### Backup an encryption key into a file in a bucket 

In [45]:
from google.cloud import storage
from google.oauth2 import service_account

# gcloud config set project  'wallet-login-45c1c'
# gsutil mb -l europe-west1 gs://gladius-backend

storage_client = storage.Client(project='wallet-login-45c1c', 
                                credentials=service_account.Credentials.from_service_account_file('../firebase-adminsdk.json'))

bucket_name = 'gladius-backend'
key_str = key.decode('utf-8')
file_name = 'gladius-key-encryption_key-ciphertext.txt'

In [34]:
#save encryption key to a bucket gs://gladius-backend

bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

blob.upload_from_string(key_str, content_type='text')


### Load an encryption key from a file in a bucket 

In [14]:
from google.cloud import storage
from google.oauth2 import service_account

# Create a storage client
storage_client = storage.Client(project='wallet-login-45c1c', 
                                credentials=service_account.Credentials.from_service_account_file('../../firebase-adminsdk.json'))

# Get the bucket and blob (file) objects
bucket_name = 'gladius-backend'
file_name = 'gladius-key-encryption_key-ciphertext.txt'
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the file contents as bytes
encryption_key = blob.download_as_bytes()



In [23]:
# Print the byte data
print(encryption_key)


b'iy1qGhJqjFTcUReSRcQ_eQ9v81vfUW8lVj4ZLn8MBQM='


Connect to KMS with SA

In [16]:
from google.cloud import kms
from google.oauth2 import service_account

# Create the credentials object from the service account file.
kms_credentials = service_account.Credentials.from_service_account_file(KMS_SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
kms_client  = kms.KeyManagementServiceClient(credentials=kms_credentials)

##### Create Gladius Key Ring - Gladius Key (if not exist in Cloud KMS)

In [17]:

# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

print(key_name)

# NEEDED ONLY FIRST TIME
# Create the key ring.
# key_ring = {}
# client.create_key_ring(request={"parent": parent, "key_ring_id": "gladius-key-ring", "key_ring": key_ring})

# Create the key.
#purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
#crypto_key = {"purpose": purpose}
#client.create_crypto_key(request={"parent": key_ring_name, "crypto_key_id": "gladius-key", "crypto_key": crypto_key})


projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key


In [24]:
key=encryption_key

In [25]:
response = kms_client.encrypt(request={"name": key_name, "plaintext": encryption_key})
encrypted_encryption_key = response.ciphertext


In [26]:
encrypted_encryption_key

b'\n$\x00\x04\xc9Y\x9bT\xabXD\x01w\xaa\xae\x04\xb9\xaf\xa4\x99;;f\xdf\xa4\x91\xc87\t#L\x12+\xa9\xe7b9C\x12U\x00\x06\x16!\xa1\xe9(D$\xb6\xa1\x80p4j\xba[\x92\x0f\xd8\xc0\x88E\xadw\x1b]]\xde\xcet\x10\x91sp\xef\x99T*\xde\xac\x0c\x1d\x19\x86,V8+\xf3\x05\x95\xca\xbf\xd4m\xc4@&kOj\x1c\r\xaf\x81\xe2R\xf4U\xe2X\x94\xa8p@s\x81\x88\xaeH\xc3\xa8\xa02'

In [16]:

# Encrypt the encryption key with Google KMS.

response = client.encrypt(request={"name": key_name, "plaintext": key})
encrypted_encryption_key = response.ciphertext


In [21]:
#encrypted_encryption_key

Use the client library to decrypt your encryption key that was encrypted with a Cloud KMS key

https://cloud.google.com/kms/docs/encrypt-decrypt

In [27]:

# Decrypt your encryption key with Google KMS.
response = client.decrypt(request={"name": key_name, "ciphertext": encrypted_encryption_key})
new_encryption_key = response.plaintext

In [29]:
new_encryption_key

b'iy1qGhJqjFTcUReSRcQ_eQ9v81vfUW8lVj4ZLn8MBQM='

In [30]:
encryption_key

b'iy1qGhJqjFTcUReSRcQ_eQ9v81vfUW8lVj4ZLn8MBQM='

## Encrypt Data in Firebase

### Connect to Firebase

In [19]:
import firebase_admin
from firebase_admin import auth, credentials, firestore


if not firebase_admin._apps:
    cred = credentials.Certificate("../../firebase-adminsdk.json")
    default_app = firebase_admin.initialize_app(cred)
# Get a reference to the firestore database
db = firestore.client()

# Get a reference to the users collection.
users_ref = db.collection("users")

# Get all documents from the users collection.
users_docs = users_ref.stream()

Create a Fernet object with your encryption key

https://www.geeksforgeeks.org/fernet-symmetric-encryption-using-cryptography-module-in-python/

In [20]:
from cryptography.fernet import Fernet

f = Fernet(encryption_key)

Use the update() method of the Firestore client to save or update the name field for a specific user document. You need to specify the document reference and a dictionary of fields and values to update. For each name field, use the encrypt() method of Fernet to encrypt it with your encryption key

In [ ]:
# Iterate over the documents and encrypt and update the name field.
for user_doc in users_docs:
  
  # Get the document data as a dictionary.
  user_data = user_doc.to_dict()

  # Get the name field from the data.
  user_name = user_data.get("name")

  # Print the name field.
  print(user_name)

  if user_name == 'test03':


    # Encrypt the name field with Fernet.
    encrypted_user_name = f.encrypt(user_name.encode())
    #print(encrypted_user_name)

    # Update the document with the encrypted name field.
    user_doc.reference.update({"glc_transaction_private_key": encrypted_user_name})
    
    # Store encrypted_encryption_key in user doc
    # user_doc.reference.update({"encrypted_encryption_key": encrypted_encryption_key})


### Decrypt private key in users/user/glc_transaction_private_key
using users/user/encrypted_encryption_key
decrypted with KMS

In [52]:
user_id = 'WUTqVpvvvcaroORmoC34HwweJJp1'  # Test03
user_ref = db.collection('users').document(user_id)
encrypted_private_key = user_ref.get().get('glc_transaction_private_key')
encrypted_encryption_key = user_ref.get().get('encrypted_encryption_key')

In [54]:
response = kms_client.decrypt(
    request={"name": key_name, "ciphertext": encrypted_encryption_key}
)
encryption_key = response.plaintext

In [55]:
cipher_suite = Fernet(encryption_key)
#encrypted_private_key = base64.b64decode(encrypted_private_key)  # If necessary

decrypted_private_key = cipher_suite.decrypt(encrypted_private_key)
decrypted_private_key

b'Bob'

In [ ]:
# encrypted_private_key was actually encrypted user_name
# glc_transaction_private_key = f.encrypt(user_name.encode())